In [7]:
#Importing read-sm-files.py
import ReadSMFIles

#reading j30.sm/j301_1.sm file
sm_file = ReadSMFIles.SMFileParser.parse_sm_file("j30.sm/j301_1.sm")

print(sm_file[1])


   renewable  nonrenewable  doubly_constrained
0          4             0                   0



Modelagem


In [ ]:
#Dados
n = 0 #numero de trabalhos
m = 0 #número de tipos de recursos



In [4]:
from Schedule import Schedule
from Task import Task
from Resource import Resource


def print_time_windows_es_ef(schedule: Schedule):
    for task in schedule.tasks:
        print(task.name, [task.earliest_start, task.earliest_finish])

schedule5 = Schedule()

d1 = Task("D1", duration=3)
d2 = Task("D2", duration=4)
d3 = Task("D3", duration=2)
d4 = Task("D4", duration=2)
d5 = Task("D5", duration=1)
d6 = Task("D6", duration=4)
d7 = Task("D7", duration=1)

d3.add_predecessors([d1])
d4.add_predecessors([d2])
d5.add_predecessors([d3])
d6.add_predecessors([d4])
d7.add_predecessors([d1])

r1 = Resource("R1", 4)

d1.add_renewable_resource(r1, 2)
d2.add_renewable_resource(r1, 3)
d3.add_renewable_resource(r1, 4)
d4.add_renewable_resource(r1, 4)
d5.add_renewable_resource(r1, 3)
d6.add_renewable_resource(r1, 2)
d7.add_renewable_resource(r1, 2)

schedule5.add_renewable_resource(r1)

schedule5.add_tasks([d2, d4, d6, d1, d3, d5, d7])

makespan = schedule5.makespan()

print_time_windows_es_ef(schedule5)
print('Schedule 5 Makespan:', makespan)

# def print_task_resources(schedule: Schedule):
#     for task in schedule.tasks:
#         print(task.name, [resource.name for resource in task.renewable_resources], [task.renewable_resources[resource] for resource in task.renewable_resources])

# print_task_resources(schedule5)

Dummy Source [0, 0]
D2 [0, 4]
D4 [4, 6]
D6 [6, 10]
D1 [6, 9]
D3 [10, 12]
D5 [12, 13]
D7 [9, 10]
Dummy Sink [13, 13]
Schedule 5 Makespan: 13
